# 第2回：転移学習と不均衡データへの挑戦

## 学習目標

- 皮膚病変画像（DermaMNIST）を用い、実務上の課題である「データの偏り」への対策を学ぶ。
- 学習済みモデルを用いた「転移学習」により、高精度なモデルを構築する。

---
## 転移学習とは

転移学習は、大規模なデータセット（ImageNetなど）で学習済みのモデルを、別のタスクに再利用する手法です。ゼロから学習するよりも少ないデータで高精度を達成できます。

## クラス不均衡とは

医療データでは、正常サンプルが多く異常サンプルが少ない「クラス不均衡」がよく発生します。そのまま学習すると、少数クラスの認識精度が低下します。

## クラス重み（class_weight）による対策

少数クラスのサンプルに大きな重みを与えることで、モデルが少数クラスも適切に学習できるようにします。

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q
sys.path.insert(0, '/tmp/MedMNIST-Exercise')
sys.modules.pop('exercise_logic', None)
import exercise_logic
exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")

# 皮膚画像データのロード（DermaMNIST）
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('dermamnist')

# クラス不均衡の可視化と重み計算
import numpy as np
from sklearn.utils import class_weight

y_train_flat = y_train.flatten()
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_flat), y=y_train_flat)
class_weights_dict = dict(enumerate(weights))
print("算出されたクラス重み:", class_weights_dict)

In [ ]:
# 転移学習モデル（MobileNetV2）を構築
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    model_type='transfer'
)

# クラス重みを適用して学習
history = model.fit(
    x_train, y_train, 
    epochs=10, 
    validation_split=0.1, 
    class_weight=class_weights_dict, 
    batch_size=32
)

In [ ]:
# 評価
exercise_logic.plot_history(history)
exercise_logic.show_evaluation_reports(model, x_test, y_test, info['label'])